# Analyze results for 3D CGAN
Feb 22, 2021

In [1]:
import     numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns
from functools import reduce

In [2]:
from ipywidgets import *

In [3]:
%matplotlib widget

In [4]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
from modules_img_analysis import *

In [5]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/5_3d_cgan/1_main_code/')
import post_analysis_pandas as post


In [6]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


### Read validation data

In [7]:
# bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,100.5,5),np.arange(100.5,300.5,20),np.arange(300.5,1000.5,50),np.array([2000])]) #bin edges to use

bins=f_transform(bins)   ### scale to (-1,1)
# ### Extract validation data
sigma_lst=[0.5,0.65,0.8,1.1]
labels_lst=range(len(sigma_lst))
bkgnd_dict={}
num_bkgnd=200

for label in labels_lst:
    fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_{0}_train_val.npy'.format(sigma_lst[label])
    print(fname)
    samples=np.load(fname,mmap_mode='r')[-num_bkgnd:][:,0,:,:]
    
    dict_val=post.f_compute_hist_spect(samples,bins)
    bkgnd_dict[str(sigma_lst[label])]=dict_val
    del samples

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.5_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.65_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.8_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_1.1_train_val.npy


## Read data

In [8]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [9]:
dict1={'64':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/',
      '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)


In [10]:
# parent_dir=u.result
parent_dir=dict1['64']
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'2021*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20210223_210217_3dcgan_predict_0.8_m2', '20210417_83…

In [11]:
result=w.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210417_83132_train


## Plot Losses

In [12]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [13]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time
371,371.0,19.0,0.199596,0.152639,0.352234,4.484216,77.067612,72.583397,-0.471264,73.284920,-0.409979,NaN,NaN,3.482550,-3.711505,-4.471950,0.786783
372,372.0,19.0,0.117810,0.153068,0.270878,3.655792,73.350098,69.694305,-0.734962,71.519379,-0.587382,NaN,NaN,3.609508,-3.052129,-3.628520,0.782176
373,373.0,19.0,0.193253,0.136837,0.330091,3.492659,76.964806,73.472145,-0.477187,69.838806,-0.781809,NaN,NaN,3.672970,-3.325343,-3.461363,0.782550
374,374.0,19.0,0.185342,0.554608,0.739950,9.619604,78.600182,68.980576,-0.854531,70.816467,-0.813236,NaN,NaN,4.111964,-0.322273,-9.619518,0.787276
375,375.0,19.0,0.899325,0.292190,1.191516,8.254401,79.202980,70.948578,-0.699103,70.226166,-0.689657,NaN,NaN,-0.166657,-8.362399,-8.254133,0.788724
376,376.0,19.0,0.175931,0.209177,0.385108,5.757905,75.844444,70.086540,-0.579995,70.458084,-0.595573,NaN,NaN,2.939071,-6.821099,-5.754425,0.782827
377,377.0,19.0,0.215505,0.151355,0.366860,3.768818,74.878403,71.109589,-0.501173,70.457008,-0.547722,NaN,NaN,4.727272,-4.803219,-3.742311,0.783514
378,378.0,19.0,0.246706,0.289783,0.536489,5.468278,75.506134,70.037857,-0.473871,71.907227,-0.454970,NaN,NaN,5.784480,-1.449222,-5.463923,0.789883
379,379.0,19.0,0.138402,0.162568,0.300970,5.768815,76.430748,70.661934,-0.643543,70.582085,-0.553609,NaN,NaN,4.719500,-4.357688,-5.765564,0.783520
380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.481384,-0.568249,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [16]:

chi=df_metrics.quantile(q=0.2,axis=0)['hist_chi']
print(chi)
df_metrics[(df_metrics['hist_chi']<=chi)&(df_metrics.epoch>1)].sort_values(by=['hist_chi']).head(10)

-0.9493226528167724


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time
196,196.0,10.0,0.255672,0.166384,0.422056,3.873117,74.567390,70.694275,-1.102573,70.489426,-1.441371,NaN,NaN,2.698059,-4.758949,-3.848714,0.794774
251,251.0,13.0,0.244835,0.149475,0.394310,5.994916,78.235580,72.240662,-1.211641,71.676880,-1.399980,NaN,NaN,4.016761,-2.543374,-5.992193,0.790036
197,197.0,10.0,0.167268,0.130952,0.298220,3.039218,74.236282,71.197067,-1.290503,70.334488,-1.386512,NaN,NaN,4.282423,-2.911951,-2.982840,0.793895
254,254.0,13.0,1.533864,0.188706,1.722569,6.870433,78.022621,71.152191,-1.342464,70.776665,-1.383763,NaN,NaN,-0.933267,-6.077430,-6.869147,0.797087
260,260.0,13.0,0.289823,1.395278,1.685101,8.081047,80.255989,72.174942,-1.265628,69.996452,-1.377975,NaN,NaN,5.306680,1.148370,-8.080603,0.792945
255,255.0,13.0,0.542144,0.196026,0.738170,1.812387,73.534164,71.721779,-1.114525,70.515656,-1.372329,NaN,NaN,1.318543,-2.358688,-1.624998,0.794924
259,259.0,13.0,0.264219,0.143264,0.407483,2.229663,73.036148,70.806488,-1.200136,70.261444,-1.360811,NaN,NaN,3.648841,-2.715520,-2.107554,0.795797
195,195.0,10.0,0.213229,0.147960,0.361190,6.326405,80.506004,74.179596,-1.134458,70.536980,-1.333313,NaN,NaN,2.082888,-5.753618,-6.323950,0.798387
277,277.0,14.0,0.178513,0.208458,0.386971,4.306799,74.513046,70.206245,-1.189903,70.895844,-1.332344,NaN,NaN,5.199939,-1.785031,-4.289037,0.803953
256,256.0,13.0,0.202685,1.434445,1.637130,8.601116,78.797638,70.196526,-1.526126,70.486290,-1.312529,NaN,NaN,3.307754,1.196354,-8.600885,0.808109


In [17]:
# display(df_metrics.sort_values(by=['hist_chi']).head(8))
# display(df_metrics.sort_values(by=['spec_chi']).head(8))

<!-- ### Read validation data -->

## Read stored chi-squares for images

In [18]:
## Get sigma list from saved files
flist=glob.glob(result_dir+'/df_processed*')
sigma_lst=[i.split('/')[-1].split('df_processed_')[-1].split('.pkle')[0] for i in flist]
sigma_lst.sort() ### Sorting is important for labels to match !!

labels_lst=np.arange(len(sigma_lst))

In [19]:
sigma_lst,labels_lst

(['0.5', '0.8', '1.1'], array([0, 1, 2]))

In [20]:
### Create a merged dataframe

df_list=[]
for label in labels_lst:
    df=pd.read_pickle(result_dir+'/df_processed_{0}.pkle'.format(str(sigma_lst[label])))
    df[['epoch','step']]=df[['epoch','step']].astype(int)
    df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting
    df_list.append(df)

for i,df in enumerate(df_list):
    df1=df.add_suffix('_'+str(i))
    # renaming the columns to be joined on
    keys=['epoch','step','img_type','label']
    rename_cols_dict={key+'_'+str(i):key for key in keys}
#     print(rename_cols_dict)
    df1.rename(columns=rename_cols_dict,inplace=True) 
    df_list[i]=df1
    
df_merged=reduce(lambda x, y : pd.merge(x, y, on = ['step','epoch','img_type','label']), df_list)

### Get sum of all 4 classes for 3 types of chi-squares
for chi_type in ['chi_1','chi_spec1','chi_1c']:
    keys=[chi_type+'_'+str(label) for label in labels_lst]
#     display(df_merged[keys].sum(axis=1))
    df_merged['sum_'+chi_type]=df_merged[keys].sum(axis=1)
del df_list



In [21]:

def f_plot_hist_spec(df,param_labels,sigma_lst,steps_list,bkg_dict,plot_type):

    img_size=64
    assert plot_type in ['hist','spec','grid','spec_relative'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))
    for par_label in param_labels:
        df=df[df.step.isin(steps_list)]
#         print(df.shape)
        idx=sigma_lst.index(par_label)
        suffix='_%s'%(idx)
        dict_bkg=bkg_dict[str(par_label)]
        
        for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
            label=row.label+'_'+str(par_label)
            if plot_type=='hist':
                x1=row['hist_bin_centers'+suffix]
                y1=row['hist_val'+suffix]
                yerr1=row['hist_err'+suffix]
                x1=f_invtransform(x1)
                
                plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
            if plot_type=='spec':
                y2=row['spec_val'+suffix]
                yerr2=row['spec_sdev'+suffix]/np.sqrt(row['num_imgs'+suffix])
                x2=np.arange(len(y2))

                plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
                plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

            if plot_type=='spec_relative':

                y2=row['spec_val'+suffix]
                yerr2=row['spec_sdev'+suffix]
                x2=np.arange(len(y2))

                ### Reference spectrum
                y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
                y=y2/y1
                ## Variance is sum of variance of both variables, since they are uncorrelated

                # delta_r= |r| * sqrt(delta_a/a)^2 +(\delta_b/b)^2) / \sqrt(N)
                yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row['num_imgs'+suffix])
                
                plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
                plt.plot(x2, y, marker=marker, linestyle=':',label=label)

            if plot_type=='grid':
                images=np.load(row.fname)[:,:,:,0]
                print(images.shape)
                f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
        ### Plot reference data
        if plot_type=='hist':
            x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
            x=f_invtransform(x)
            plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
            plt.title('Pixel Intensity Histogram')
            plt.xscale('symlog',linthreshx=50)
        
        if plot_type=='spec':
            y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
            x=np.arange(len(y))
            plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
            plt.title('Spectrum')
            plt.xlim(0,img_size/2)

        if plot_type=='spec_relative':
            plt.axhline(y=1.0,color='k',linestyle='-.')
            plt.title("Relative spectrum")
            plt.xlim(0,img_size/2)
            plt.ylim(0.5,2)

    if plot_type in ['hist','spec']:     
        plt.yscale('log')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})


# f_plot_hist_spec(df_merged,[sigma_lst[-1]],sigma_lst,[best_step[0]],bkgnd_dict,'hist')

### Slice best steps

In [22]:
def f_slice_merged_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',label='all',params_lst=[0,1,2],head=10,epoch_range=[0,None],use_sum=True,display_flag=False):
    ''' View dataframe after slicing
    '''

    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ######### Apply cutoff to keep reasonable chi1 and chispec1
    #### Add chi-square columns to use
    chi_cols=[]
    if use_sum: ## Add sum chi-square columns
        for j in ['chi_1','chi_spec1','chi_1c']: chi_cols.append('sum_'+j)
        
    if label=='all': ### Add chi-squares for all labels
        for j in ['chi_1','chi_spec1','chi_1c']:
            for idx,i in enumerate(params_lst): chi_cols.append(j+'_'+str(idx))
    else: ## Add chi-square for specific label
        assert label in params_lst, "label %s is not in %s"%(label,params_lst)
        label_idx=params_lst.index(label)
        print(label_idx)
        for j in ['chi_1','chi_spec1','chi_1c']: chi_cols.append(j+'_'+str(label_idx))
#     print(chi_cols)
    q_dict=dict(df_merged.quantile(q=cutoff,axis=0)[chi_cols])
    # print(q_dict)
    strg=['%s < %s'%(key,q_dict[key]) for key in chi_cols ]
    query=" & ".join(strg)
    # print(query)
    df=df.query(query)
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
    chis=[i for i in df_merged.columns if 'chi' in i]
    col_list=['label']+chis+['epoch','step']
    if (col_mode=='short'): 
        col_list=['label']+[i for i in df_merged.columns if i.startswith('sum')]
        col_list=['label']+chi_cols
    df2=df1.head(head)[col_list]
    
    if display_flag: display(df2) # Display df
    
    return df2

# f_slice_merged_df(df_merged,cutoff=0.3,sort_col='sum_chi_1',label=0.65,params_lst=[0.5,0.65,0.8,1.1],use_sum=True,head=2000,display_flag=False,epoch_range=[7,None])

In [23]:
cols_to_sort=np.unique([i for i in df_merged.columns for j in ['chi_1_','chi_spec1_'] if ((i.startswith(j)) or (i.startswith('sum')))])

w=interactive(f_slice_merged_df,df=fixed(df_merged),
cutoff=widgets.FloatSlider(value=0.3, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
use_sum=widgets.Checkbox(value=True),
label=ToggleButtons(options=['all']+sigma_lst), params_lst=fixed(sigma_lst),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=cols_to_sort
)
display(w)

interactive(children=(FloatSlider(value=0.3, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [24]:
df_sliced=w.result
# df_sliced

In [25]:
best_step=[]

best_step.append(f_slice_merged_df(df_merged,cutoff=0.9,sort_col='sum_chi_1',label='all',use_sum=True,head=4,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)
best_step.append(f_slice_merged_df(df_merged,cutoff=0.9,sort_col='sum_chi_spec1',label='all',use_sum=True,head=4,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)
best_step.append(f_slice_merged_df(df_merged,cutoff=0.9,sort_col='sum_chi_1c',label='all',use_sum=True,head=2,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)

# best_step.append([46669,34281])
best_step=np.unique([i for j in best_step for i in j])
print(best_step)
best_step

[150 220 280 320 350 360 370 380]


array([150, 220, 280, 320, 350, 360, 370, 380])

In [26]:
# best_step=[6176]
# best_step= [32300, 35810, 36020, 37030, 38640, 42480, 43850]

# best_step=np.arange(40130,40135).astype(int)

In [27]:
df_best=df_merged[df_merged.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(8, 61)
[(7, 150), (11, 220), (14, 280), (16, 320), (18, 350), (18, 360), (19, 370), (19, 380)]


In [28]:
col_list=['label']+[i for i in df_merged.columns if i.startswith('sum')]

df_best[col_list]

,label,sum_chi_1,sum_chi_spec1,sum_chi_1c
14,7-150,0.565518,4608.142089,0.0
21,11-220,0.594530,887.058494,0.0
27,14-280,0.507987,6175.550881,0.0
31,16-320,0.417079,1092.575674,0.0
34,18-350,0.509773,887.085584,0.0
35,18-360,0.457323,862.883473,0.0
36,19-370,1.991047,751.086385,0.0
37,19-380,0.539789,612.233053,0.0


### Interactive plot

In [29]:
interact_manual(f_plot_hist_spec,df=fixed(df_merged),
                param_labels=SelectMultiple(options=sigma_lst),sigma_lst=fixed(sigma_lst),
                steps_list=SelectMultiple(options=best_step),
                bkg_dict=fixed(bkgnd_dict),plot_type=ToggleButtons(options=['hist','spec','grid','spec_relative']))

interactive(children=(SelectMultiple(description='param_labels', options=('0.5', '0.8', '1.1'), value=()), Sel…

<function __main__.f_plot_hist_spec(df, param_labels, sigma_lst, steps_list, bkg_dict, plot_type)>

In [ ]:
# ### Check deterministic
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# epoch,step=0,230
# f1='20210113_185116_cgan_predict_0.65_m2/images/gen_img_label-0.5_epoch-{0}_step-{1}.npy'.format(epoch,step)
# f2='20210113_092234_cgan_predict_0.65_m2/images/gen_img_label-0.5_epoch-{0}_step-{1}.npy'.format(epoch,step)

# epoch,step=3,20
# f1='20210114_191648_nb_test/images/gen_img_label-0.5_epoch-{0}_step-{1}.npy'.format(epoch,step)
# f2='20210114_193009_nb_test/images/gen_img_label-0.5_epoch-{0}_step-{1}.npy'.format(epoch,step)

# a1=np.load(main_dir+f1)
# a2=np.load(main_dir+f2)
# # print(a1.shape,a2.shape)

# print(np.mean(a1),np.mean(a2))
# print(np.max(a1),np.max(a2))

### Delete unwanted stored models
(Since deterministic runs aren't working well )

In [113]:
# fldr='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210119_134802_cgan_predict_0.65_m2/models'
fldr=result_dir
print(fldr)
flist=glob.glob(fldr+'/models/checkpoint_*.tar')
len(flist)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210227_050213_3dcgan_predict_0.65_m2


3742

In [31]:
# # Delete unwanted stored images
# for i in flist:
#     try:
#         step=int(i.split('/')[-1].split('_')[-1].split('.')[0])
#         if step not in best_step:
# #             print("Found")
# #             print(step)
#             os.remove(i)
#             pass
#         else: 
#             print(step)
# #             print(i)
#     except Exception as e:
# #         print(e)
# #         print(i)
#         pass

In [118]:
best_step

array([27600, 30950, 42620, 43830, 50440, 58320, 58330, 59500, 64670,
       65600])

In [34]:
np.load('/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.5_train_val.npy',mmap_mode='r').shape



(70656, 1, 64, 64, 64)

In [33]:
! du -hs  /global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.5_train_val.npy

70G	/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.5_train_val.npy
